# Chains with multiple inputs


In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?\n\nBecause he wanted to be polyunsaturated!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "german"})

{'input': 'a parrot',
 'language': 'german',
 'text': 'Warum hat der Papagei den Bus verpasst?\n\nWeil er noch schnell einen Vogel hatte!'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to german: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="german_translation"
)

In [5]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "german_translation"],
)

overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})

{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': 'I recently ordered the Pizza Salami from this restaurant and I have to say, it was absolutely awful. The pizza arrived cold and soggy, with barely any toppings on it. The salami was barely noticeable and tasted like it had been sitting out for hours.\n\nThe crust was overcooked and tasted like cardboard. I was extremely disappointed with the quality of the pizza, especially considering the price I paid for it.\n\nOverall, I would not recommend ordering the Pizza Salami from this restaurant. Save your money and go somewhere else for a better pizza experience.',
 'comment': "I had a similar experience when I ordered the Pizza Salami from this restaurant. The quality was definitely not up to par and I was very disappointed. I agree that the toppings were sparse and the salami was barely there.\n\nI also found the crust to be overcooked and lacking in flavor. It's a shame because I had heard good things about this pl